In [1]:

!pip install sentencepiece

     |████████████████████████████████| 1.2MB 23.6MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')
# data를 저장할 폴더 입니다. 환경에 맞게 수정 하세요.
data_dir = "/content/drive/My Drive/Capstone2/"

Mounted at /content/drive


In [3]:
import os
import gzip
import pandas as pd
import sentencepiece as spm
import shutil

In [4]:
for f in os.listdir(data_dir):
  print(f)

bias_dataset_train.txt
bias_dataset_test.txt
text_classification_with_transformer
text_classification_with_transformer_korean
Generate_vocab.ipynb


In [5]:
in_file = f"{data_dir}/kowiki.csv.gz"
out_file = f"{data_dir}/kowiki.txt"
SEPARATOR = u"\u241D"

if not os.path.isfile(out_file):
    df = pd.read_csv(in_file, sep=SEPARATOR, engine="python", compression='gzip')
    with open(out_file, "w") as f:
      for index, row in df.iterrows():
        f.write(row["text"]) # title 과 text를 중복 되므로 text만 저장 함
        f.write("\n\n\n\n") # 구분자

shutil.copy(out_file, "kowiki.txt")

'kowiki.txt'

In [6]:
corpus = "kowiki.txt"
prefix = "kowiki"
vocab_size = 8000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 기타 추가 토큰

for f in os.listdir("."):
  print(f)
shutil.copy("kowiki.model", f"{data_dir}/kowiki.model")
shutil.copy("kowiki.vocab", f"{data_dir}/kowiki.vocab")

.config
drive
kowiki.vocab
kowiki.model
kowiki.txt
sample_data


'/content/drive/My Drive/Capstone2//kowiki.vocab'

In [7]:
vocab_file = f"{data_dir}/kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

lines = [
  "겨울이 되어서 날씨가 무척 추워요.",
  "이번 성탄절은 화이트 크리스마스가 될까요?",
  "겨울에 감기 조심하시고 행복한 연말 되세요."
]
for line in lines:
  pieces = vocab.encode_as_pieces(line)
  ids = vocab.encode_as_ids(line)
  print(line)
  print(pieces)
  print(ids)
  print()

겨울이 되어서 날씨가 무척 추워요.
['▁겨울', '이', '▁되어', '서', '▁날', '씨', '가', '▁무', '척', '▁추', '워', '요', '.']
[3091, 3588, 601, 3602, 683, 4014, 3599, 108, 4193, 206, 3958, 3760, 3590]

이번 성탄절은 화이트 크리스마스가 될까요?
['▁이번', '▁성', '탄', '절', '은', '▁화', '이트', '▁크리스', '마', '스가', '▁될', '까', '요', '?']
[3224, 86, 3967, 3923, 3604, 264, 669, 1970, 3664, 780, 1450, 3794, 3760, 4245]

겨울에 감기 조심하시고 행복한 연말 되세요.
['▁겨울', '에', '▁감', '기', '▁조', '심', '하', '시', '고', '▁행', '복', '한', '▁연', '말', '▁되', '세', '요', '.']
[3091, 3591, 212, 3605, 53, 3832, 3596, 3613, 3600, 234, 3871, 3603, 61, 3823, 445, 3682, 3760, 3590]

